In [4]:
def pagerank(G, alpha=0.85, personalization=None, max_iter=100, tol=1.0e-6, nstart=None, weight='weight', dangling=None):
    if len(G) == 0:
        return {}

    if not G.is_directed():
        D = G.to_directed()
    else:
        D = G

    # Create a copy in (right) stochastic form
    W = nx.stochastic_graph(D, weight=weight)
    N = W.number_of_nodes()

    # Choose fixed starting vector if not given
    if nstart is None:
        x = dict.fromkeys(W, 1.0 / N)
    else:
        # Normalized nstart vector
        s = float(sum(nstart.values()))
        x = dict((k, v / s) for k, v in nstart.items())

    if personalization is None:
        # Assign uniform personalization vector if not given
        p = dict.fromkeys(W, 1.0 / N)
    else:
        missing = set(G) - set(personalization)
        if missing:
            raise NetworkXError('Personalization dictionary must have a value for every node. Missing nodes %s' % missing)
        s = float(sum(personalization.values()))
        p = dict((k, v / s) for k, v in personalization.items())

    if dangling is None:
        # Use personalization vector if dangling vector not specified
        dangling_weights = p
    else:
        missing = set(G) - set(dangling)
        if missing:
            raise NetworkXError('Dangling node dictionary must have a value for every node. Missing nodes %s' % missing)
        s = float(sum(dangling.values()))
        dangling_weights = dict((k, v/s) for k, v in dangling.items())

    dangling_nodes = [n for n in W if W.out_degree(n, weight=weight) == 0.0]

     # power iteration: make up to max_iter iterations
    for _ in range(max_iter):
        xlast = x
        x = dict.fromkeys(xlast.keys(), 0)
        danglesum = alpha * sum(xlast[n] for n in dangling_nodes)
        for n in x:
            # this matrix multiply looks odd because it is
            # doing a left multiply x^T=xlast^T*W
            for nbr in W[n]:
                x[nbr] += alpha * xlast[n] * W[n][nbr][weight]
            x[n] += danglesum * dangling_weights[n] + (1.0 - alpha) * p[n]

        # check convergence, l1 norm
        err = sum([abs(x[n] - xlast[n]) for n in x])
        if err < N*tol:
            return x
    raise NetworkXError('Pagerank: power iteration failed to converge in %d iterations.' % max_iter)

In [5]:
import networkx as nx

In [6]:
G = nx.barabasi_albert_graph(60, 41)
pr = nx.pagerank(G, 0.4)

In [12]:
print(pr)

{0: 0.028060461857347126, 1: 0.012760328222114365, 2: 0.012571576490056354, 3: 0.012366587669864355, 4: 0.012965582893731753, 5: 0.012951284690860026, 6: 0.012764378213312161, 7: 0.012957826818110446, 8: 0.012748583123464888, 9: 0.012991961940040165, 10: 0.013773607979725315, 11: 0.012553519325363431, 12: 0.012165580851467983, 13: 0.012988060384730812, 14: 0.012963733748069578, 15: 0.012564062428221393, 16: 0.012544819896366913, 17: 0.01296828672853895, 18: 0.013156264951205017, 19: 0.012368259434404817, 20: 0.012971299215159384, 21: 0.013562386671019451, 22: 0.012357109991672402, 23: 0.013365583938759935, 24: 0.012757474348559256, 25: 0.012965969029621292, 26: 0.012970905007161996, 27: 0.012367189413632747, 28: 0.012769227835381484, 29: 0.013566869773826387, 30: 0.013773607979725315, 31: 0.012966147220171904, 32: 0.012967722590243435, 33: 0.01234077415229502, 34: 0.013562386671019451, 35: 0.013366715656979524, 36: 0.012954799179360214, 37: 0.013367486259910879, 38: 0.01277272725798291